# Workflow to Download, Screen, and Record GNSS observations locally

- **Original code authored by:** David Bekaert, Heresh Fattahi, Eric Fielding, and Zhang Yunjun with 
Extensive modifications by Adrian Borsa and Amy Whetter and other NISAR team members 2022

- **Derived code made by Grace Bato, Jinwoo Kim, and Simran Sangha ** October, 2024


<div class="alert alert-warning">
The initial cells with hardcoded values an setup section (<b>Prep A</b> section) should be run at the start of the notebook. And all subsequent sections NEED to be run in order.
</div>



In [1]:
# Parameters for papermill

### Choose a site from the 'sites' dictionary found 2 cells down
site = '8882'
frameID_str = str(site).zfill(5)    # forced to have 5 digit for frameID string 
work_dir = './'
gnss_csv_dir = 'GNSS_record'    # location of GNSS record csv files

calval_sites_csv = 'validation_data/DISP-S1_CalVal_sites.csv'

#Set GNSS Parameters
gps_completeness_threshold = 0.9    #0.9  #percent of data timespan with valid GNSS epochs
gnss_thr_eq = 11.0

# [optional] manually remove additional stations
gnss_to_remove = [] # ['ANG5', 'ANG6', 'TSFT', 'TXAG', 'TXH4', 'TXHA', 'TXHE', 'TXHS', 'TXP5', 'TXRS', 'UHKD', 'UHL1']

In [2]:
# Load standard library packages
import json
import os
import shutil
import warnings
import zipfile
from datetime import datetime as dt
from io import BytesIO
from pathlib import Path

# Load third-party packages
import geopandas as gpd
import matplotlib
import matplotlib.colors
import numpy as np
import pandas as pd
import requests
import ruptures as rpt  # for detecting significant steps (conda install conda-forge::ruptures)
from matplotlib import pyplot as plt
from scipy import signal
from scipy.optimize import curve_fit
from scipy.signal import find_peaks

# Configure matplotlib
plt.rcParams.update({'font.size': 12})

# Load local application-specific packages
from mintpy.objects import gnss
from mintpy.objects.gnss import search_gnss

# Suppress specific warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Load functions to access and apply GNSS corrections

In [ ]:
def detect_step_functions(dates, time_series, penalty=1, threshold=0.1, window_size=10):
    """
    Detect significant step functions and stair-like features in a time series with corresponding dates.
    
    Args:
    dates (list): List of dates corresponding to the time series data.
    time_series (list or numpy.array): The input time series data.
    penalty (float): Penalty term for the change point detection algorithm. Default is 1.
    threshold (float): The minimum relative change to consider as a significant step. Default is 0.1 (10%).
    window_size (int): Size of the window for local trend analysis. Default is 10.
    
    Returns:
    list: List of tuples (date, index, step_type) where significant step functions or stairs are detected.
    """
    # Convert input to numpy array if it's a list
    if isinstance(time_series, list):
        time_series = np.array(time_series)
    
    # Ensure dates and time_series have the same length
    if len(dates) != len(time_series):
        raise ValueError("The lengths of dates and time_series must be the same.")
    
    # Fit the change point detection model
    model = rpt.Pelt(model="rbf").fit(time_series.reshape(-1, 1))
    
    # Find the change points
    change_points = model.predict(pen=penalty)
    
    # Calculate first differences
    diff = np.diff(time_series)
    
    # Find peaks in the absolute differences (potential steps)
    peaks, _ = find_peaks(np.abs(diff), height=np.std(diff)*2)
    
    # Combine change points and peaks
    all_points = sorted(set(change_points[1:]).union(set(peaks)))
    
    # Filter significant steps and detect stair-like features
    significant_features = []
    for point in all_points:
        before = np.mean(time_series[max(0, point-window_size):point])
        after = np.mean(time_series[point:min(point+window_size, len(time_series))])
        
        relative_change = (after - before) / abs(before)
        
        if abs(relative_change) > threshold:
            # Check for stair-like feature
            if point + window_size < len(time_series):
                next_window = np.mean(time_series[point+window_size:min(point+2*window_size, len(time_series))])
                if abs((next_window - after) / after) < threshold/2:
                    significant_features.append((dates[point], point, "stair"))
                else:
                    significant_features.append((dates[point], point, "step"))
            else:
                significant_features.append((dates[point], point, "step"))
    
    return significant_features

In [ ]:
# Setup GNSS screening classes
def read_UNR_gnss_step(url):
    """ 
    reading steps.txt file in UNR database
    : including info about equipment changes and earthquake occurrences
    
    """

    if url.startswith('http'):
        # fetch the content from the URL
        response = requests.get(url)
        content = response.text
    else:
        with open(url, 'r') as local_file:
            content = local_file.read()

    # Initialize lists to store data
    data = []
    
    # Process the content line by line
    for line in content.split('\n'):
        if line.strip() == '':
            continue  # Skip empty lines
        
        parts = line.split()
        if len(parts) < 3:
            continue  # Skip lines with insufficient data
        
        code = int(parts[2])
        
        if code == 1 and len(parts) >= 4:
            data.append(parts[:4])
        elif code == 2 and len(parts) >= 7:
            data.append(parts[:7])

    # Create DataFrame
    columns = [
        'site',
        'step_date',
        'code',
        'type_threshold',
        'distance_to_epicenter',
        'magnitude',
        'eventID'
    ]
    df = pd.DataFrame(data, columns=columns)
    
    # Convert data types
    df['code'] = df['code'].astype(int)
    
    # Convert column2 to datetime
    df['step_date'] = pd.to_datetime(df['step_date'], format='%y%b%d', errors='coerce')
    
    for col in df.columns:
        if col not in ['code', 'step_date']:  # Skip code and date columns
            df[col] = pd.to_numeric(df[col], errors='ignore')
    
    return df

# Define a flexible model function that handles velocity and a variable number of steps
def tot_model(t, v, *A_steps):
    # Linear velocity component
    displacement = v * t
    # Add step changes using Heaviside function if step times are provided
    for i, A in enumerate(A_steps):
        displacement += A * np.heaviside(t - step_times_numeric[i], 1)
    return displacement

# Subtract the step components if steps exist, leaving the velocity component
def step_model(t, *A_steps):
    step_component = 0
    for i, A in enumerate(A_steps):
        step_component += A * np.heaviside(t - step_times_numeric[i], 1)
    return step_component

## Set workspace

In [ ]:
### Define list of requirements
## Static for OPERA Cal/Val requirements, do not touch!

# specify GNSS source for validation
gnss_source = 'UNR'
print(f'Searching for all GNSS stations from source: {gnss_source}')
print(f'May use any of the following supported sources: {gnss.GNSS_SOURCES}')
GNSS = gnss.get_gnss_class(gnss_source)
unr_data_holdings = 'http://geodesy.unr.edu/NGLStationPages/DataHoldings.txt'
gnss_steps_url = 'http://geodesy.unr.edu/NGLStationPages/steps.txt'

In [ ]:
################# Set Directories ##########################################
print('\nCurrent directory:', os.getcwd())

if 'work_dir' not in locals():
    work_dir = Path.cwd()

work_dir = os.path.abspath(work_dir)    # absolute path       
print("Work directory:", work_dir)
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)  # Change to Workdir   

gnss_csv_dir = f'{work_dir}/{gnss_csv_dir}'
if not os.path.exists(gnss_csv_dir):
    os.mkdir(gnss_csv_dir)

plot_dir = f'{work_dir}/gnss_correction_plots'
if not os.path.exists(plot_dir):
    os.mkdir(plot_dir)

orig_gnss_ts = f'{plot_dir}/gnss_ts_before_corrections'
if not os.path.exists(orig_gnss_ts):
    os.mkdir(orig_gnss_ts)

bad_gnss_dir = f'{plot_dir}/stations_with_extrasteps'
if not os.path.exists(bad_gnss_dir):
    os.mkdir(bad_gnss_dir)

############################################################################
### List of OPERA DISP-S1 Cal/Val Sites for secular requirements:
if os.path.exists(calval_sites_csv):
    sites_df = pd.read_csv(calval_sites_csv) 
else:
    raise FileNotFoundError(f"The file {calval_sites_csv} does not exist.")  

display(sites_df)

secular_available_sites = sites_df['site'].values

## Table of Contents:
<a id='secular_TOC'></a>

<hr/>

[**Prep. Environment Setup**](#env_prep)

[**1. Find Collocated GNSS Stations**](#find_gnss)

[**2. Get GNSS Position Time Series and apply corrections as needed**](#apply_gnss_corrections)

<hr/>

<a id='env_prep'></a>
## Prep. Environment Setup
Setup your environment for processing data

In [ ]:
os.chdir(work_dir)  # move to working directory 

<a id='find_gnss'></a>
## 1. Find Collocated GNSS Stations

Reading DISP-S1 frame

In [ ]:
# Extracting DISP-S1 frame 
# URL of the ZIP file containing the JSON file
repo_zip_url = 'https://github.com/opera-adt/burst_db/releases/download/v0.5.0/frame-geometries-simple-0.5.0.geojson.zip'

# Download the ZIP file
response = requests.get(repo_zip_url)
zip_data = BytesIO(response.content)

# Extract the JSON file from the ZIP archive
with zipfile.ZipFile(zip_data, 'r') as zip_ref:
    # Assuming your JSON file is named 'data.json' within the ZIP
    json_data = zip_ref.read('frame-geometries-simple-0.5.0.geojson')

# Load the JSON data
data = json.loads(json_data.decode('utf-8')) # ['features']

gdf = gpd.GeoDataFrame.from_features(data)
gdf['frameID'] = pd.json_normalize(data["features"])["id"].values     # ID in 'feature' of geojson is added
gdf = gdf.set_crs(epsg=4326)        # defining crs of geopandas table

gdf.head()

In [ ]:
gdf_select = gdf[gdf['frameID']==int(site)]
display(gdf_select)

# geometry of extracted frameID
geom = gdf_select['geometry'].values[0].bounds

DISP_region_geo = (geom[1],geom[3],geom[0],geom[2])

Query for GNSS sites that coincide with the specified DISP frame

In [ ]:
# get analysis metadata from DISP-S1 velocity file
#!#disp_s1_metadata = readfile.read_attribute(vel_file)
#!#start_date = disp_s1_metadata.get('START_DATE', None)
#!#end_date = disp_s1_metadata.get('END_DATE', None)
start_date = '20140101'
end_date = '20241021'
start_date_gnss = dt.strptime(start_date, "%Y%m%d")
end_date_gnss = dt.strptime(end_date, "%Y%m%d")

# search for collocated GNSS stations
site_names, site_lats_wgs84, site_lons_wgs84 = gnss.search_gnss(SNWE=DISP_region_geo,
                                                                start_date=start_date,
                                                                end_date=end_date,
                                                                source=gnss_source)

site_names = [str(stn) for stn in site_names]
print("Initial list of {} stations used in analysis:".format(len(site_names)))
print(site_names)

Query and pass antennae pattern corrections for the GPS stations

In [ ]:
# Get steps and corrections for each station
gnss_steps_df = read_UNR_gnss_step(gnss_steps_url) # reading UNR step file

# gnss_steps_ant_df: table for equipment changes
# equipment (antenna, receiver or firmware) changes
gnss_steps_ant_df = gnss_steps_df[gnss_steps_df['code']==1].reset_index(drop=True)
gnss_steps_ant_df = gnss_steps_ant_df.iloc[:, :4]
_cols = gnss_steps_ant_df.columns.tolist()
_cols[3] = 'type'
gnss_steps_ant_df.columns = _cols

# gnss_steps_ant_df: table for earthquake occurrences
# earthquake event
gnss_steps_eq_df = gnss_steps_df[gnss_steps_df['code']==2].reset_index(drop=True)
_cols = gnss_steps_eq_df.columns.tolist()
_cols[3] = 'threshold_distance'
gnss_steps_eq_df.columns = _cols

# filtering equipment changed based on dates
# print('Type of equipment changes: ', gnss_steps_ant_df['type'].unique())
# list of problematic equipment changes; can be expanded
prob_changes = [
    'Antenna_detached_from_monument', 'Equipment_Site_Change', 
    'Antenna_Changed', 'Antenna_change', 'Antenna_Code_Changed'
]
# if the type of equipment changes belong to problematic list
gnss_steps_ant_df = gnss_steps_ant_df[gnss_steps_ant_df['type'].isin(prob_changes)]
gnss_steps_ant_df = gnss_steps_ant_df[(gnss_steps_ant_df['step_date'] >= start_date_gnss) & \
    (gnss_steps_ant_df['step_date']<= end_date_gnss)].reset_index(drop=True)

# filter gnss stations based on earthquake magnitude, distance, dates
gnss_steps_eq_df = gnss_steps_eq_df[gnss_steps_eq_df['magnitude'] > gnss_thr_eq]
gnss_steps_eq_df = gnss_steps_eq_df[(gnss_steps_eq_df['step_date'] >= start_date_gnss) & \
    (gnss_steps_eq_df['step_date'] <= end_date_gnss)].reset_index(drop=True)

# just pass antennae pattern corrections
gnss_steps_merged = gnss_steps_ant_df[['site', 'step_date']]
# Drop duplicates
gnss_steps_merged = gnss_steps_merged.drop_duplicates().reset_index(drop=True)

<a id='apply_gnss_corrections'></a>
## 2. Get GNSS Position Time Series and apply corrections as needed

Search for documented equipment-related steps in the time-series and solve for/apply step adjustments.

Also, filter out stations that have undocumented, large artifacts.

Record the final list of stations to be used for Cal/Val in a CSV file, and also generate quality control plots for the stations that are rejected.

In [ ]:
# get daily position solutions for GNSS stations
use_stn = []  #stations to keep
bad_stn = []  #stations to toss

# track latlon coordinates for UTM grids
use_lats_keepwgs84 = [] 
use_lons_keepwgs84 = []
bad_lats_keepwgs84 = [] 
bad_lons_keepwgs84 = []

# expected tenv column names
gnss_col_names = [
    'site', 'YYMMMDD', 'yyyy.yyyy', 'MJD', 'week', 'd', 'reflon', 'e0(m)', 
    'east(m)', 'n0(m)', 'north(m)', 'u0(m)', 'up(m)', 'ant(m)', 
    'sig_e(m)', 'sig_n(m)', 'sig_u(m)', 'corr_en', 'corr_eu', 'corr_nu', 
    'latitude(deg)', 'longitude(deg)', 'height(m)'
]

fpath = os.path.join(work_dir,f'GNSS-{gnss_source}')
for counter, stn in enumerate(site_names):
    gps_obj = GNSS(site = stn,
                   data_dir = fpath)
    gps_obj.open(print_msg=False)
    
    # open file with pandas
    fname = f'{fpath}/{stn}.tenv3'
    df_gnss = pd.read_csv(fname, names=gnss_col_names, header=0, sep=r'\s+')
    df_gnss['datetimes'] = pd.to_datetime(df_gnss['YYMMMDD'], format='%y%b%d')

    # filter by dates
    range_days = (end_date_gnss - start_date_gnss).days
    df_gnss = df_gnss[(df_gnss['datetimes'] >= start_date_gnss) & \
                  (df_gnss['datetimes'] <= end_date_gnss)]
    # Reindex the filtered DataFrame
    df_gnss = df_gnss.reset_index(drop=True)
    gnss_count = len(df_gnss)
    dates = df_gnss['datetimes'].to_list()

    # select GNSS stations based on data completeness
    if (range_days * gps_completeness_threshold <= gnss_count) \
        and stn not in gnss_to_remove:
        record_original_ts = f'{orig_gnss_ts}/{stn}.tenv3'
        # get GNSS steps corresponding to antennae pattern corrections & EQs
        step_times = gnss_steps_merged[
            gnss_steps_merged['site'] == stn
        ]['step_date'].to_list()
        if len(step_times) > 0:
            #!# temporarily override correction and pass to bad stations
            # track bad stations
            bad_stn.append(stn)
            bad_lats_keepwgs84.append(site_lats_wgs84[counter])
            bad_lons_keepwgs84.append(site_lons_wgs84[counter])
            # Quarantine the original TS to a separate directory
            record_bad_ts = f'{bad_gnss_dir}/{stn}.tenv3'
            shutil.move(fname, record_original_ts)
            continue
            # reference_date = dates[0]
            # gnss_times_numeric = np.array([(d - reference_date).days for d in dates])
            # step_times_numeric = [(d - reference_date).days for d in step_times]

            # # Initial guess for the parameters: velocity, and step amplitudes
            # initial_guess = [1] + [1] * len(step_times)

            # # loop through e, n, and u displacement
            # disp_dict = {}
            # disp_dict['east(m)'] = df_gnss['east(m)'].to_list()
            # disp_dict['north(m)'] = df_gnss['north(m)'].to_list()
            # disp_dict['up(m)'] = df_gnss['up(m)'].to_list()

            # # initiate plot
            # fig, ax = plt.subplots(3, 1, figsize=(12, 15))
            # for ind, comp in enumerate(disp_dict.keys()):
            #     # filter for comp and subtracted median
            #     gnss_dis = disp_dict[comp]
            #     gnss_dis_median = np.nanmedian(gnss_dis)
            #     gnss_dis -= gnss_dis_median
            #     gnss_intercept = gnss_dis[0]
            #     gnss_dis -= gnss_intercept
        
            #     # Perform curve fitting, handling the case of no steps (linear fit only)
            #     params, covariance = curve_fit(tot_model,
            #         gnss_times_numeric,
            #         gnss_dis,
            #         p0=initial_guess)

            #     # Extract the fitted velocity and step amplitudes
            #     v_fit = params[0]
            #     A_steps_fit = params[1:]

            #     # solve for step disp
            #     step_component = step_model(gnss_times_numeric, *A_steps_fit)

            #     # Isolate the velocity component
            #     gnss_dis_substep = gnss_dis - step_component

            #     # add back median
            #     gnss_dis += gnss_intercept
            #     gnss_dis += gnss_dis_median
            #     gnss_dis_substep += gnss_intercept
            #     gnss_dis_substep += gnss_dis_median

            #     # overwrite array with correction
            #     df_gnss[comp] = gnss_dis_substep

            #     # plot results
            #     ax[ind].axhline(color='grey',linestyle='dashed', linewidth=2)
            #     ax[ind].scatter(dates, gnss_dis, s=2**2,
            #                     color='gray', alpha=0.5,
            #                     label="Raw GNSS Daily Positions")
            #     ax[ind].scatter(dates, gnss_dis_substep, s=2**2,
            #                     color='blue',
            #                     label="Corrected GNSS Daily Positions")
            #     ymin = np.min([gnss_dis, gnss_dis_substep])
            #     ymax = np.max([gnss_dis, gnss_dis_substep])
            #     ax[ind].vlines(step_times, ymin=ymin, ymax=ymax,
            #                    color='red', linestyle='--', alpha=0.7)
            #     ax[ind].set_ylim(ymin, ymax) 
            #     # axis format
            #     ax[ind].set_ylabel(comp)
            #     ax[ind].legend()

            # # Add a supertitle to the entire figure
            # fig.suptitle(f'Station Name: {stn}')
            # # Adjust layout
            # plt.tight_layout()
            # # Save plot
            # plt.savefig(os.path.join(plot_dir, f'GNSS_station{stn}_correction.jpg'))

            # # Move the original TS before saving the updated one
            # shutil.move(fname, record_original_ts)

            # # Save updated TS to file
            # df_gnss.drop(columns=['datetimes'], inplace=True)
            # df_gnss.to_csv(fname, sep=' ', index=False)
        ##################################################################
        ##################################################################
        # Filtering stations based on significant steps in TS
        # not captured in the UNR record
        dis_stn = np.sqrt(df_gnss['east(m)'].to_numpy()**2 + \
                          df_gnss['north(m)'].to_numpy()**2 +  \
                          df_gnss['up(m)'].to_numpy()**2)
        detected_steps = detect_step_functions(dates, dis_stn)
        if len(detected_steps) != 0:
            # plotting bad stations in case of further investigation
            fig, ax = plt.subplots(figsize=(15,10))
            ax.plot(dates, dis_stn,'.')
            ax.set_title(f'Station: {stn}')
            ax.set_xlabel('Dates', fontsize=16, fontweight='bold')
            ax.set_ylabel('GNSS measurements', fontsize=16, fontweight='bold')
            ax.set_xlim([start_date_gnss, end_date_gnss])
            ax.grid(axis='x')
            fig.savefig(f'{bad_gnss_dir}/{stn}_plot.png', bbox_inches='tight', dpi=300)
            plt.close()

            # Quarantine the original TS to a separate directory
            record_bad_ts = f'{bad_gnss_dir}/{stn}.tenv3'
            shutil.move(fname, record_original_ts)

            # track bad stations
            bad_stn.append(stn)
            bad_lats_keepwgs84.append(site_lats_wgs84[counter])
            bad_lons_keepwgs84.append(site_lons_wgs84[counter])
        else:
            # track good stations
            use_stn.append(stn)
            use_lats_keepwgs84.append(site_lats_wgs84[counter])
            use_lons_keepwgs84.append(site_lons_wgs84[counter])
    else:
        # Quarantine the original TS to a separate directory
        record_bad_ts = f'{bad_gnss_dir}/{stn}.tenv3'
        shutil.move(fname, record_original_ts)

        # track bad stations
        bad_stn.append(stn)
        bad_lats_keepwgs84.append(site_lats_wgs84[counter])
        bad_lons_keepwgs84.append(site_lons_wgs84[counter])

site_names = use_stn
site_lats_wgs84 = use_lats_keepwgs84
site_lons_wgs84 = use_lons_keepwgs84

print("\nFinal list of {} stations used in analysis:".format(len(site_names)))
print(site_names)
print("List of {} stations removed from analysis".format(len(bad_stn)))
print(bad_stn)

Save final kept stations to csv lookup file

In [ ]:
gnss_csv_file = f'{gnss_csv_dir}/frame{site}.csv'
df_gnss = pd.DataFrame({'site': site_names,
                        'lat': use_lats_keepwgs84,
                        'lon': use_lons_keepwgs84})
df_gnss.to_csv(gnss_csv_file, index=False)
del df_gnss

Also track final rejected stations to csv file for debugging purposes

In [ ]:
rejected_gnss_csv_file = f'{gnss_csv_dir}/frame{site}_rejectedstations.csv'
df_gnss = pd.DataFrame({'site': bad_stn,
                        'lat': bad_lats_keepwgs84,
                        'lon': bad_lons_keepwgs84})
df_gnss.to_csv(rejected_gnss_csv_file, index=False)
del df_gnss